<h1><center>Seismology: Lab 2

In [1]:
from plotly import __version__, tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

# import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches
from scipy import io
from IPython.core.display import display, HTML
import scipy.stats as stats
import seaborn as sns
#sns.set(color_codes=True)
#plt.rcParams["patch.force_edgecolor"] = True
%matplotlib inline

In [164]:
HTML('''
<script>
    var code_show=true; //true -> hide code at first

    function code_toggle() {
        $('div.prompt').hide(); // always hide prompt

        if (code_show){
            $('div.input').hide();
        } else {
            $('div.input').show();
        }
        code_show = !code_show
    }
    $( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()">[Toggle Code]</a>
''')

In [2]:
data = pd.read_csv("lab01_data.csv")

lat = np.array(data['latitude'])
lon = np.array(data['longitude'])
depth = np.array(data['depth'])
Mw = np.array(data['mag'])

y = np.array(data["time"])
n = len(y)  # number of data points

# convert time yyyy-mm-dd-h-m-s to continuous format
year = np.zeros(n)
mm = np.zeros(n)
dd = np.zeros(n)
h = np.zeros(n)
m = np.zeros(n)
s = np.zeros(n)

for i in range(len(y)):
    year[i] = float(y[i][0:4])
    mm[i] = float(y[i][5:7])
    dd[i] = float(y[i][8:10])
    h[i] = float(y[i][11:13])
    m[i] = float(y[i][14:16])
    s[i] = float(y[i][17:23])

# time = from (1900-2017) in years
time = year + (mm + (dd + (h + (m + s/60)/60)/24)/30)/12 

<h3>1. Temporal variation of earthquakes</h3>  
  


In [3]:
trace = go.Scatter(x = time, y = Mw, mode='markers+text',
                   marker = dict(color = 'white', 
                                 line = dict(width=1, color = 'blue'))
                  )

data1 = [trace]
layout = go.Layout(title='Earthquake Magnitude (Mw) vs Year',
                   xaxis= dict(title='Year'),
                   yaxis =dict(title='Moment Magnitude (Mw)'),
                   showlegend = False)

layout.update(dict(shapes = [
                    {
                        'type': 'rect',
                        'x0': '2004.5',
                        'x1': '2005.4',
                        'y0': '3.9',
                        'y1': '6.2',
                        'line': {'color': 'red'},
                    }]
                  )
             )
layout.update(dict(annotations = [go.Annotation(
                    text = "Main Earthquake Sequence",
                   x=2005, y=6.2)
                                 ]))

fig = go.Figure(data=data1, layout=layout)
iplot(fig)

<h3>2. Time period of the sequence 

It is approximately from 06/01/2004 to 04/01/2005. The mainshock is magnitude Mmax = 6.1.

<h3>3. New catalogue with Mw range [0 - 6.1] 

In [7]:
data2 = pd.read_csv("lab02_data.csv")

lat2 = np.array(data2['latitude'])
lon2 = np.array(data2['longitude'])
depth2 = np.array(data2['depth'])
Mw2 = np.array(data2['mag'])

y2 = np.array(data2["time"])
n2 = len(y2)  # number of data points

# convert time yyyy-mm-dd-h-m-s to continuous format
year2 = np.zeros(n2)
mm2 = np.zeros(n2)
dd2 = np.zeros(n2)
h2 = np.zeros(n2)
m2 = np.zeros(n2)
s2 = np.zeros(n2)

for i in range(len(y2)):
    year2[i] = float(y2[i][0:4])
    mm2[i] = float(y2[i][5:7])
    dd2[i] = float(y2[i][8:10])
    h2[i] = float(y2[i][11:13])
    m2[i] = float(y2[i][14:16])
    s2[i] = float(y2[i][17:23])

# time = from (1900-2017) in years
time2 = year2 + (mm2 + (dd2 + (h2 + (m2 + s2/60)/60)/24)/30)/12 

<h3>4. Plot MFD 

In [36]:
values, base = np.histogram(Mw2, bins=30)
cumulative = np.cumsum(values[::-1])

base = base[np.nonzero(base)]

trace2 = go.Scatter(x = base, y = values, name='Non-Cumulative',
                    mode='markers',
                   marker = dict(color = 'blue'))
                    
trace3 = go.Scatter(x = base, y = cumulative[::-1], name='Cumulative',
                    mode='markers',
                   marker = dict(color = 'red'))
data2 = [trace2, trace3]
layout = go.Layout(title='Magnitude frequency distribution (Bin size = 0.04)',
                   xaxis= dict(title='Magnitude (Mw)'),
                   yaxis =dict(title='Frequency'))

fig = go.Figure(data=data2, layout=layout)
iplot(fig)

In [37]:
trace2 = go.Scatter(x = base, y = values, name='Non-Cumulative',
                    mode='markers',
                   marker = dict(color = 'blue'))
                    
trace3 = go.Scatter(x = base, y = cumulative[::-1], name='Cumulative',
                    mode='markers',
                   marker = dict(color = 'red'))
data2 = [trace2, trace3]
layout = go.Layout(title='Magnitude frequency distribution',
                   xaxis= dict(title='Magnitude (Mw)'),
                   yaxis =dict(title='Frequency', type='log'))

fig = go.Figure(data=data2, layout=layout)
iplot(fig)

The MFD is linear for most part except for very low and very high magnitudes, where it is saturated. Yes, this region has the Magnitude of completeness Mc ~ 1.0.

<h3>5. b-values of the region

In [114]:
base_f = np.array([base[x] for x in range(len(base)) if 5>=base[x]>1])
val_f = np.array([values[x] for x in range(len(values)) if 5>=base[x]>1])
cum_f = np.array([cumulative[x] for x in range(len(values)) if 5>=base[x]>1])

slope_cum, intercept, r_value, p_value, std_err = stats.linregress(base_f, np.log10(cum_f[::-1]))
slope_nc, intercept, r_value, p_value, std_err = stats.linregress(base_f[np.nonzero(val_f)], 
                                                    np.log10(val_f[np.nonzero(val_f)]))

The b-value for non cumulative MFD is -0.79 and the b-value for the cumulative MFD is -0.73. I have calculated it using linear least squares L2 minimization, and I have only included the data from $1\le M_w\le5$ so that only the linear part of the curve is fit.

<h3>6 and 7. Omori's law

In [144]:
mag = np.array([Mw2[x] for x in range(len(Mw2)) if Mw2[x]>1])
days = np.array([time2[x] for x in range(len(Mw2)) if Mw2[x]>1])
#days = days*12*30
days = (days - 2004.83)*12*30

nt = np.zeros(len(days))

for i in range(len(days)):
    nt[i] = sum(days>=i*10)

nt = nt[np.nonzero(nt)]
x = np.arange(1, len(nt[np.nonzero(nt)])+1)*10

In [154]:
trace3 = go.Scatter(x = x, y = nt, mode='markers+text',
                   marker = dict(color = 'blue', 
                                 line = dict(width=1, color = 'blue')),
                    name = 'data'
                  )

trace4 = go.Scatter(x = x, y = (38788/x), mode='lines',
                   marker = dict(color = 'red', 
                                 line = dict(width=1, color = 'blue')),
                    name='best fit curve'
                  )
data3 = [trace3, trace4]
layout = go.Layout(title='No. of earthquakes in days',
                   xaxis= dict(title='Days since mainshock'),
                   yaxis =dict(title='Number of aftershocks'))


fig = go.Figure(data=data3, layout=layout)
iplot(fig)

If $c = 0$ and $p = 1$, the modified Omori's law becomes $n(t) = \frac{k}{t}$. In the above graph, if we assume $t$ in days, the value of $k$ is given by the product of $n(t)$ and $t$. The above graph is a rectangular hyperbola, and therefore $k$ is constant. Thus, the mean value of $k$ is approximately $38788$ for this unnormalized graph. The curve fits the data well for $p = 1$.

<h3> 8. Changing constants in Omori's law

In [160]:
K = 38788
c = 0.05; p = 1;
omori1 = K/(x+c)**1
omori2 = K/(x+c)**1.5
omori0 = K/(x+c)**0.5

In [163]:
trace3 = go.Scatter(x = x, y = nt, mode='markers+text',
                   marker = dict(color = 'blue', 
                                 line = dict(width=1, color = 'blue')),
                    name = 'data'
                  )

trace4 = go.Scatter(x = x, y = omori0, mode='lines',
                   marker = dict(color = 'red'),
                    name='curve with p = 0.5'
                  )
trace5 = go.Scatter(x = x, y = omori1, mode='lines',
                   marker = dict(color = 'green', 
                                 line = dict(width=1, color = 'blue')),
                    name='curve with p = 1'
                  )
trace6 = go.Scatter(x = x, y = omori2, mode='lines',
                   marker = dict(color = 'black', 
                                 line = dict(width=1, color = 'blue')),
                    name='curve with p = 1.5'
                  )
data3 = [trace3, trace4, trace5, trace6]
layout = go.Layout(title='No. of earthquakes in days',
                   xaxis= dict(title='Days since mainshock'),
                   yaxis =dict(title='Number of aftershocks'))


fig = go.Figure(data=data3, layout=layout)
iplot(fig)

$p = 1$ gives the best fit curve for this data as well.